# Cointoss Estimation using Classical Game Theory

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
# global variables
PROB_HEAD = 0.63
ROUNDS_PER_SIM = 1000
NUM_SIMS = 20
SEED = 42

## Payoff Structure

In [3]:
def payoff_function(guess: str, outcome: str) -> int:
    if guess == "H" and outcome == "H":
        return 2
    elif guess == "T" and outcome == "T":
        return 1
    else:
        return 0


## Game Simulation Setup

In [4]:
def simulate_game(
    strategy_p1: callable,
    strategy_p2: callable,
    p_head: float,
    rounds: int,
    seed: int = None,
) -> dict:

    if seed is not None:
        np.random.seed(seed)

    # initialize record-keeping
    guesses_p1 = []
    guesses_p2 = []
    payoffs_p1 = []
    payoffs_p2 = []

    # flip coin
    for round_idx in range(rounds):
        outcome = "H" if np.random.rand() < p_head else 'T'

        # get each player's guess based on strategy
        # players have access to each others guesses and payoffs
        guess1 = strategy_p1(round_idx, guesses_p1, guesses_p2, payoffs_p1, payoffs_p2)
        guess2 = strategy_p2(round_idx, guesses_p2, guesses_p1, payoffs_p2, payoffs_p1)

        # compute individual payoffs
        payoff1 = payoff_function(guess1, outcome)
        payoff2 = payoff_function(guess2, outcome)

        # update history
        guesses_p1.append(guess1)
        guesses_p2.append(guess2)
        payoffs_p1.append(payoff1)
        payoffs_p2.append(payoff2)

    score_p1 = sum(payoffs_p1)
    score_p2 = sum(payoffs_p2)

    return {
        "guesses_p1": guesses_p1,
        "guesses_p2": guesses_p2,
        "payoffs_p1": payoffs_p1,
        "payoffs_p2": payoffs_p2,
        "final_scores": (score_p1, score_p2),
    }

## Game Strategies

In [5]:
def strategy_always_heads(
    round_idx, own_guesses, opp_guesses, own_payoffs, opp_payoffs
):
    return "H"


def strategy_always_tails(
    round_idx, own_guesses, opp_guesses, own_payoffs, opp_payoffs
):
    return "T"


def strategy_random(round_idx, own_guesses, opp_guesses, own_payoffs, opp_payoffs):
    return "H" if np.random.rand() < 0.5 else "T"


def strategy_adaptive(round_idx, own_guesses, opp_guesses, own_payoffs, opp_payoffs):
    """
    Example 'adaptive' strategy:
      - If last round's payoff was 0, switch guess (from H->T or T->H).
      - Otherwise, repeat the same guess as last round.
      - If it's the first round (round_idx=0), guess 'H' by default.
    """
    if round_idx == 0:
        return "H"
    else:
        last_payoff = own_payoffs[-1]
        last_guess = own_guesses[-1]
        if last_payoff == 0:
            # Switch guess
            return "H" if last_guess == "T" else "T"
        else:
            # Keep the same guess
            return last_guess


def strategy_copy_opponent(
    round_idx, own_guesses, opp_guesses, own_payoffs, opp_payoffs
):
    """
    Example 'copy-opponent' strategy:
      - Always guess whatever the opponent guessed last round.
      - First round defaults to 'H'.
    """
    if round_idx == 0:
        return "H"
    else:
        return opp_guesses[-1]

## Running the Game

In [6]:
if __name__ == "__main__":
    # Always Heads vs. Adaptive

    np.random.seed(42)  # For reproducibility

    result = simulate_game(
        strategy_p1=strategy_always_heads,
        strategy_p2=strategy_adaptive,
        p_head=PROB_HEAD,
        rounds=ROUNDS_PER_SIM,
        seed=SEED,
    )

    score_p1, score_p2 = result["final_scores"]
    print("Final scores after 1000 rounds:")
    print("Player 1 (always heads):", score_p1)
    print("Player 2 (adaptive):", score_p2)
    print()

    # run multiple sims, store results, and compare averages
    n_sims = NUM_SIMS
    sum_scores_p1 = 0
    sum_scores_p2 = 0
    for s in range(n_sims):
        res = simulate_game(
            strategy_p1=strategy_always_heads,
            strategy_p2=strategy_adaptive,
            p_head=PROB_HEAD,
            rounds=ROUNDS_PER_SIM,
            seed=s,  # different seeds each time
        )
        sp1, sp2 = res["final_scores"]
        sum_scores_p1 += sp1
        sum_scores_p2 += sp2

    print(f"Averaged over {n_sims} simulations:")
    print("Player 1 average score:", sum_scores_p1 / n_sims)
    print("Player 2 average score:", sum_scores_p2 / n_sims)

Final scores after 1000 rounds:
Player 1 (always heads): 1292
Player 2 (adaptive): 977

Averaged over 20 simulations:
Player 1 average score: 1271.9
Player 2 average score: 934.85


In [7]:
def plot_cumulative_scores(payoffs_p1, payoffs_p2, strategy_1, strategy_2):
    pay1 = np.array(payoffs_p1)
    pay2 = np.array(payoffs_p2)

    cum_scores_p1 = np.cumsum(pay1)
    cum_scores_p2 = np.cumsum(pay2)

    # Use f-strings to name the columns
    col_p1 = f"Player 1 {strategy_1}"
    col_p2 = f"Player 2 {strategy_2}"

    df = pd.DataFrame(
        {
            "Round": np.arange(1, len(pay1) + 1),
            col_p1: cum_scores_p1,
            col_p2: cum_scores_p2,
        }
    )

    fig = px.line(
        df,
        x="Round",
        y=[col_p1, col_p2],  # columns in the DataFrame
        labels={"value": "Cumulative Score", "variable": "Player"},
        title=f"Cumulative Scores Over Rounds: {strategy_1} vs {strategy_2}",
    )

    fig.show()

In [8]:
plot_cumulative_scores(
    payoffs_p1=result["payoffs_p1"],
    payoffs_p2=result["payoffs_p2"],
    strategy_1="Always Heads",
    strategy_2="Adaptive",
)

In [9]:
if __name__ == "__main__":
    # Always Heads vs. Always Tails

    np.random.seed(42)  # For reproducibility

    result = simulate_game(
        strategy_p1=strategy_always_heads,
        strategy_p2=strategy_always_tails,
        p_head=PROB_HEAD,
        rounds=ROUNDS_PER_SIM,
        seed=SEED,
    )

    score_p1, score_p2 = result["final_scores"]
    print("Final scores after 1000 rounds:")
    print("Player 1 (always heads):", score_p1)
    print("Player 2 (always tails):", score_p2)
    print()

    # run multiple sims, store results, and compare averages
    n_sims = NUM_SIMS
    sum_scores_p1 = 0
    sum_scores_p2 = 0
    for s in range(n_sims):
        res = simulate_game(
            strategy_p1=strategy_always_heads,
            strategy_p2=strategy_always_tails,
            p_head=PROB_HEAD,
            rounds=ROUNDS_PER_SIM,
            seed=s,  # different seeds each time
        )
        sp1, sp2 = res["final_scores"]
        sum_scores_p1 += sp1
        sum_scores_p2 += sp2

    print(f"Averaged over {n_sims} simulations:")
    print("Player 1 average score:", sum_scores_p1 / n_sims)
    print("Player 2 average score:", sum_scores_p2 / n_sims)

Final scores after 1000 rounds:
Player 1 (always heads): 1292
Player 2 (always tails): 354

Averaged over 20 simulations:
Player 1 average score: 1271.9
Player 2 average score: 364.05
